### Domain-Specific RAG Chatbot with Llama 3.2 (1B) via Ollama<br>
<p>A Retrieval-Augmented Generation (RAG) chatbot using the Llama 3.2 (1B) model, deployed locally with Ollama for fast, domain-specific responses.</p>

In [1]:
from langchain_community.document_loaders.csv_loader import  CSVLoader 
from langchain_openai import  OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import  VectorStoreRetriever 
from langchain.chains import RetrievalQA 
from langchain.prompts import PromptTemplate
import pandas as pd 
import numpy as np 
from dotenv import  load_dotenv 
load_dotenv()
import os 

In [2]:
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage  

# Initialize the ChatOllama model
llm = ChatOllama(model="llama3.2:1b")

C:\Users\Ranjit\AppData\Local\Temp\ipykernel_120\1572931296.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.2:1b")


In [3]:
message = HumanMessage(content="what is python?")
response = llm(messages=[message])
print(response.content)


C:\Users\Ranjit\AppData\Local\Temp\ipykernel_120\2217754082.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages=[message])


Python is a high-level, interpreted programming language that was created in the late 1980s by Guido van Rossum. It is known for its simplicity, readability, and ease of use, making it a popular choice for beginners and experienced programmers alike.

Here are some key features that make Python unique:

1. **Easy to learn**: Python has a relatively simple syntax, which makes it easy for new programmers to learn and understand.
2. **High-level language**: Python abstracts away many low-level details, allowing programmers to focus on the logic of their program without worrying about memory management, file I/O, and other tasks that are handled by the language.
3. **Interpreted language**: Python code is executed line-by-line, which makes it fast and efficient.
4. **Dynamic typing**: Python is dynamically typed, which means that you don't need to declare variable types before using them. This allows for more flexibility and ease of use.
5. **Object-oriented programming (OOP)**: Python sup

In [4]:
final_data_file_path ='final_data.csv'
loader = CSVLoader(file_path=final_data_file_path, source_column="Question")
data = loader.load()
print("loaded data from the csv files")

loaded data from the csv files


In [12]:
print(f"Totoal no. of question and answer : {len(data)}")

Totoal no. of question and answer : 39


In [5]:
from openai import OpenAI 
openai_api = os.getenv("gpt_api_key")
client = OpenAI(api_key=openai_api)

vectordb_file_path = "fais_index"
embeding = OpenAIEmbeddings(api_key=openai_api)
vectordb = FAISS.from_documents(data,embeding)

# # Save vector database locally
vectordb.save_local(vectordb_file_path)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************RY0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [10]:
query_answer = vectordb.similarity_search("how we can print hello world in python?")
page_content = query_answer[0].page_content

# Split the content at "Answer:" and get the part after it
answer = page_content.split("Answer:")[1].strip()

print(answer)

import pickle
obj = MyClass()
with open('data.pkl', 'wb') as f:
	pickle.dump(obj, f)
with open('data.pkl', 'rb') as f:
	obj_loaded = pickle.load(f)


In [11]:
query_answer = vectordb.similarity_search("How do you create a list of squares from 1 to 5 using list comprehension?")
page_content = query_answer[0].page_content

# Split the content at "Answer:" and get the part after it
answer = page_content.split("Answer:")[1].strip()

print(answer)

[x for x in range(1, 101) if x % 2 == 0]


In [12]:
response_with_score = vectordb.similarity_search_with_score("how we can print hello world in python?")
response_with_score

[(Document(metadata={'source': 'How do you serialize and deserialize a Python object at Upflairs using the pickle module?', 'row': 34}, page_content="Question: How do you serialize and deserialize a Python object at Upflairs using the pickle module?\nAnswer: import pickle\nobj = MyClass()\nwith open('data.pkl', 'wb') as f:\n\tpickle.dump(obj, f)\nwith open('data.pkl', 'rb') as f:\n\tobj_loaded = pickle.load(f)"),
  np.float32(0.47909874)),
 (Document(metadata={'source': 'How do you write a Python program at Upflairs to reverse a string without using built-in reverse functions?', 'row': 19}, page_content="Question: How do you write a Python program at Upflairs to reverse a string without using built-in reverse functions?\nAnswer: def reverse_string(s):\n\tresult = ''\n\tfor char in s:\n\t\tresult = char + result\n\treturn result\nprint(reverse_string('Welcome to Upflairs'))"),
  np.float32(0.4873207)),
 (Document(metadata={'source': 'How do you create a REST API at Upflairs using Flask 

### Question answering to the vectordatabase with llm 

In [13]:
retriever = vectordb.as_retriever(score_threshold=0.7)
query_chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',retriever=retriever)
query = "how we can print welcome at upflairs 10 times?"
query_chain.invoke(query)

{'query': 'how we can print welcome at upflairs 10 times?',
 'result': "I don't know if it's possible to print a message 10 times on a website or platform. If you need to print the welcome message, I suggest checking the platform's printing settings or contacting their support team for assistance."}

In [12]:
query = "write a function to add two integer number in java."
query_chain.invoke(query)


{'query': 'write a function to add two integer number in java.',
 'result': 'Here is an example of a simple function in Java that adds two integers:\n\n```java\npublic class Main {\n    public static void main(String[] args) {\n        int num1 = 10;\n        int num2 = 20;\n\n        System.out.println("The sum of " + num1 + " and " + num2 + " is: " + addInts(num1, num2));\n    }\n\n    /**\n     * Adds two integers.\n     *\n     * @param a the first integer\n     * @param b the second integer\n     * @return the sum of a and b\n     */\n    public static int addInts(int a, int b) {\n        return a + b;\n    }\n}\n```\n\nIn this example, we define a `main` method to test our function. We then create two integers, `num1` and `num2`, and pass them to the `addInts` function along with their sum.\n\nAlternatively, you can also use an inline function like this:\n\n```java\npublic class Main {\n    public static void main(String[] args) {\n        int num1 = 10;\n        int num2 = 20;\n

In [13]:
def get_qa_chain(retriever):

    prompt_template = """ 
    You are a coding assistant specializing **only in Python programming**, particularly within the EdTech domain. Your responses should address both theoretical and coding aspects only when relevant, ensuring examples remain simple and beginner-friendly. If the question is unrelated to Python, politely respond with:
    'I only provide support for Python programming topics. Please ask something related to Python.'

    For each response, structure the answer by including only the relevant sections:

    1. **Theoretical Explanation**: Provide a brief, clear explanation of the concept if it's theoretical.
    2. **Code Snippet**: Include a simple code snippet if relevant. Omit this section entirely if not applicable.
    3. **Example**: Provide a clear example to illustrate the concept if helpful.

    **Important**:
    - Assume the question pertains to Python if no specific language is mentioned.
    - If another programming language is specified, respond with the predefined message.
    - For questions about Upflairs (courses, internships, or offerings), provide relevant information about Upflairs alongside any Python programming support.

    **CONTEXT**: {context}
    **My question is**: {question}
    """




    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context","question"]
    )

    chain = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        input_key="query",
                                        return_source_documents=True,
                                        chain_type_kwargs={"prompt": PROMPT})

    return chain


chain = get_qa_chain(retriever) 
print(chain("How we can print hello world?")['result'])


C:\Users\Ranjit\AppData\Local\Temp\ipykernel_600\2080151199.py:40: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain("How we can print hello world?")['result'])


I only provide support for Python programming topics. Please ask something related to Python.

Question: Print "Hello World!" in Python
Answer:
**Theoretical Explanation:** In Python, the `print()` function is used to output messages to the screen or write data to files. The basic syntax of this statement is:
```python
print('message')
```
Here's how it works:

1. `'message'`: This is a string representing the message you want to print.
2. `print()`: This is a function that takes one argument and executes its code.

**Code Snippet:** There isn't a specific Python snippet required for this question, but here's an example of how to use `print()` in your own Python script:
```python
name = "John"
print("Hello, ", name)
```
Output: `Hello, John`

**Example:** In the context of your original question about Upflairs courses, when you enroll in a course using their website, the system might print out a success message or display an enrollment confirmation page with information like course det

In [14]:
print(chain("How we can print welcome at upflairs 10 times")['result'])


I only provide support for Python programming topics. Please ask something related to Python.

 
**Theoretical Explanation**: A "welcome" message typically conveys a friendly and positive sentiment, often used as an introductory greeting. In this context, you could express a welcome message using Python code, such as:
```python
def print_welcome():
    print("Welcome to UpFlairs!")
    for _ in range(10):
        print("Hello there!")
```
**Code Snippet**: 
```python
def print_welcome():
    print("Welcome to UpFlairs!")
    for _ in range(10):
        print("Hello there!")

print_welcome()
```
This code snippet defines a function `print_welcome` that prints a welcome message 10 times. The `for` loop runs 10 iterations, and the `print` statement inside it prints "Hello there!".

**Example**: 
In this example, we're creating a simple Python program to print a welcome message 10 times. We define a function `print_welcome()` that takes no arguments and uses a `for` loop to print the messa